# Juliaで100本ノック(76-100)

## 準備

In [ ]:
ENV["COLUMNS"]=240  # 描画する表の列数を増やす
ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）

using Pkg

Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("LibPQ")
Pkg.add("StatsBase")
Pkg.add("ScikitLearn")
Pkg.add("MLDataPattern")
Pkg.add("CSV")

using DataFrames
using DataFramesMeta
using LibPQ
using StatsBase
using Statistics
using Dates
using Random
using ScikitLearn
using MLDataPattern
using CSV

In [ ]:
@sk_import preprocessing: (LabelBinarizer, StandardScaler, MinMaxScaler)

## SQLとの接続

In [ ]:
host = "db"
port = ENV["PG_PORT"]
database = ENV["PG_DATABASE"]
user = ENV["PG_USER"]
password = ENV["PG_PASSWORD"]
dsl = "postgresql://$user:$password@$host:$port/$database"
conn = LibPQ.Connection(dsl)

df_customer = DataFrame(execute(conn, "select * from customer"))
df_category = DataFrame(execute(conn, "select * from category"))
df_product = DataFrame(execute(conn, "select * from product"))
df_receipt = DataFrame(execute(conn, "select * from receipt"))
df_store = DataFrame(execute(conn, "select * from store"))
df_geocode = DataFrame(execute(conn, "select * from geocode"));

# 本編

### 076

In [ ]:
# ScikitLearnに渡すためにいったんArrayに変換、そのあとまたDataFrameに戻す。
_, arr_tmp = CrossValidation.train_test_split(Array(df_customer), test_size=0.1, stratify=df_customer.gender_cd)
df_tmp = DataFrame(arr_tmp)
rename!(df_tmp, names(df_customer));

@linq df_tmp |>
    select(:customer_id, :gender_cd) |>
    groupby(:gender_cd) |>
    combine(:customer_id => length) |>
    orderby(:gender_cd)

### 077

In [ ]:
# これまでにやったものをつなぎ合わせるだけ
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_ss = zscore(:amount_sum)) |>
    where(abs.(:amount_ss) .>= 3) |>
    orderby(:customer_id) |>
    first(10)

### 078

In [ ]:
# 少し行数が多くなるが素直に計算する
df_tmp = @linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum);

fourquantile = nquantile(df_tmp.amount_sum, 4)
delta = iqr(df_tmp.amount_sum)*1.5
lowerbound = fourquantile[2] - delta
upperbound = fourquantile[4] + delta

@linq df_tmp |>
    where((:amount_sum .< lowerbound) .| (:amount_sum .> upperbound)) |>
    orderby(:customer_id) |>
    first(10)

### 079

In [ ]:
Dict(zip(names(df_product), sum.(eachcol(ismissing.(df_product)))))

### 080

In [ ]:
df_product_1 = dropmissing(df_product);
println(string("削除前: ", nrow(df_product)))
println(string("削除後: ", nrow(df_product_1)))

### 081

In [ ]:
df_product_2 = copy(df_product)

# あまり必要はないがちょっと凝ったことをしてみる。列の中身をin-placeに書き換えたいのでindexingに!を使ってコピーを回避する。
# そのうえで、毎回df_product_2[!, colname]と書くのはまどろっこしいのでこれを指すポインタを定義。Ref(var)がC言語の&varに相当。
# C言語で*pointer = ...とポインタの中身を操作するのはpointer[] = ...と書く。
function p081!(df)
    for colname in [:unit_price, :unit_cost]
        p_col = Ref(df[!, colname])  # 参照を取る場合は!を使う
        filling = round(mean(skipmissing(p_col[])))
        p_col[] .= coalesce.(p_col[], filling)
    end
end

@time p081!(df_product_2)
Dict(zip(names(df_product), sum.(eachcol(ismissing.(df_product_2)))))

### 082

In [ ]:
# 本当は全ての処理をfunctionの中に書いたほうがパフォーマンスの点でベター。
# ここではこの数問のみでしかやってないものの、まとまった量のデータを処理する場合は注意。
df_product_3 = copy(df_product)

function p082!(df)
    for colname in [:unit_price, :unit_cost]
        p_col = Ref(df[!, colname])
        filling = round(median(skipmissing(p_col[])))
        p_col[] .= coalesce.(p_col[], filling)
    end
end

@time p082!(df_product_3)
Dict(zip(names(df_product), sum.(eachcol(ismissing.(df_product_3)))))

### 083

In [ ]:
df_tmp1 = @linq df_product |>
    select(:category_small_cd, :unit_price) |>
    dropmissing() |>
    groupby(:category_small_cd) |>
    combine(:unit_price => median)
cd_to_price = Dict([(each.category_small_cd, each.unit_price_median) for each in eachrow(df_tmp1)]);

In [ ]:
# 上のpriceとまとめてdataframeにしてもいいところだが、もしmissingの入りかたが独立の場合は結果が正しくなくなってしまうので別々に処理
df_tmp2 = @linq df_product |>
    select(:category_small_cd, :unit_cost) |>
    dropmissing() |>
    groupby(:category_small_cd) |>
    combine(:unit_cost => median)
cd_to_cost = Dict([(each.category_small_cd, each.unit_cost_median) for each in eachrow(df_tmp2)]);

In [ ]:
fill_price(cd, price) = ismissing(price) ? cd_to_price[cd] : price
fill_cost(cd, cost) = ismissing(cost) ? cd_to_cost[cd] : cost

In [ ]:
df_product_4 = @linq df_product |>
    transform(unit_price = fill_price.(:category_small_cd, :unit_price)) |>
    transform(unit_cost = fill_cost.(:category_small_cd, :unit_cost));
Dict(zip(names(df_product), sum.(eachcol(ismissing.(df_product_4)))))

### 084

In [ ]:
# まずは年を取り出してunstack、さらにmissingを埋める
df_tmp = @linq df_receipt |>
    transform(year = :sales_ymd .÷ 10000) |>
    transform(is2019 = :year .== 2019) |>
    groupby([:customer_id, :is2019]) |>
    combine(:amount => sum) |>
    unstack(:customer_id, :is2019, :amount_sum)
rename!(df_tmp, [:customer_id, :before2019, :in2019])
df_tmp = coalesce.(df_tmp, 0)

# 割合を計算してフィルタ
df_tmp2 = @linq df_tmp |>
    transform(total = :before2019 .+ :in2019) |>
    transform(ratio = :in2019 ./ :total) |>
    where(:ratio .> 0) |>
    orderby(:customer_id)

# 非顧客の行を消すためdf_customerをjoin
@linq innerjoin(df_tmp2, @select(df_customer, :customer_id), on=:customer_id) |>
    first(10)

In [ ]:
Dict(zip(names(df_product), sum.(eachcol(ismissing.(df_tmp2)))))

### 085

In [ ]:
# いったん平均を取るために使う列だけで処理してから再度joinする
df_tmp = @linq innerjoin(df_customer, @select(df_geocode, :postal_cd, :longitude, :latitude), on=:postal_cd) |>
    select(:customer_id, :longitude, :latitude) |>
    groupby(:customer_id) |>
    combine(:longitude => mean, :latitude => mean);

In [ ]:
df_customer_1 = innerjoin(df_customer, df_tmp, on=:customer_id);
first(df_customer_1, 3)

### 086

In [ ]:
# 緯度経度がDecimal型のままではdeg2radに入らないのでFloatにキャスト。ほかはPython版の式を移植しただけ。
function calc_distance(x1, y1, x2, y2)
    x1, y1, x2, y2 = deg2rad(Float64(x1)), deg2rad(Float64(y1)), deg2rad(Float64(x2)), deg2rad(Float64(y2))  
    L = 6371 * acos(sin(y1)*sin(y2)
                    + cos(y1)*cos(y2)*cos(x1 - x2))
    return L
end

In [ ]:
# 店舗の緯度経度を計算
# joinの左右で列名が違う場合はこのようにon = :left => :rightとする
df_tmp = @linq innerjoin(@select(df_customer_1, :customer_id, :application_store_cd), df_store, on=:application_store_cd=>:store_cd) |>
    select(:customer_id,
            store_lat = :latitude,
            store_lon = :longitude);

In [ ]:
# 顧客と店舗の緯度経度をjoin
df_tmp2 = innerjoin(@select(df_customer_1, :customer_id, customer_lat = :latitude_mean, customer_lon = :longitude_mean),
          df_tmp,
          on=:customer_id);

In [ ]:
# 距離を計算
df_tmp3 = @byrow! df_tmp2 begin
    @newcol distance::Array{Float64}
    :distance = calc_distance(:customer_lat, :customer_lon, :store_lat, :store_lon)
end;

@linq df_tmp3 |>
    orderby(:customer_id) |>
    first(3)

### 087

In [ ]:
# 別テーブルから売り上げを集計
df_amount_sum = @linq df_receipt |>
    groupby(:customer_id) |>
    combine(:amount => sum);

In [ ]:
df_customer_u = @linq leftjoin(df_customer, df_amount_sum, on=:customer_id) |>
    transform(amount_sum = coalesce.(:amount_sum, 0)) |>
    sort([order(:amount_sum, rev=true), order(:customer_id)]) |>
    groupby([:customer_name, :postal_cd]) |>
    combine([(each => first) for each in names(df_customer)[begin:end]]) |>  # All => firstと書きたいができないのでそれぞれの列にタプルを作る
    select(Not(["customer_name", "postal_cd"])) |>  # ここはSymbolではだめという……
    orderby(:customer_id_first)
rename!(df_customer_u, [each for each in names(df_customer)]);

In [ ]:
println(nrow(df_customer))
println(nrow(df_customer_u))

### 088

In [ ]:
# いったんmakeuniqueで列名を変更させてからリネーム。pandas同様にDictで特定の列だけリネームもできる。
df_customer_n = innerjoin(df_customer, df_customer_u[:, [:customer_id, :customer_name, :postal_cd]], on=[:customer_name, :postal_cd], makeunique=true)
rename!(df_customer_n, Dict([(:customer_id_1, :integration_id)]));

In [ ]:
first(df_customer_n, 3)

### 089

In [ ]:
df_tmp = innerjoin(df_customer, df_receipt, on=:customer_id);

In [ ]:
# ScikitLearnに渡すためにいったんArrayに変換、そのあとまたDataFrameに戻す。
arr_train, arr_test = CrossValidation.train_test_split(Array(df_tmp), test_size=0.2, random_state=71);

In [ ]:
df_train = DataFrame(arr_train)
df_test = DataFrame(arr_test);

In [ ]:
println(nrow(df_train) / nrow(df_tmp))
println(nrow(df_test) / nrow(df_tmp))

### 090

In [ ]:
df_tmp = @linq df_receipt |>
    transform(month = :sales_ymd .÷ 100) |>
    groupby(:month) |>
    combine(:amount => sum) |>
    orderby(:month);

In [ ]:
# スライスを指定するときに1-originでboth-sides inclusiveなことに注意する
function split_data(df::DataFrame; start_offset::Int64, train_size=12, test_size=6, slide_window=6)
    train_start = 1 + start_offset*6
    train_end = train_start + train_size - 1
    test_start = train_end + 1
    test_end = test_start +test_size - 1
    return df[train_start:train_end, :], df[test_start:test_end, :]
end

In [ ]:
df_train_1, df_test_1 = split_data(df_tmp, start_offset=0)
df_train_2, df_test_2 = split_data(df_tmp, start_offset=1)
df_train_3, df_test_3 = split_data(df_tmp, start_offset=2);

### 091

In [ ]:
# 常に1を返すlambda式で定数を割り当て（もっとスマートな方法ありそう）
df_ifpurchased = @linq df_receipt |>
    select(:customer_id) |>
    unique() |>
    transform(if_purchased = (x -> 1).(:customer_id));

In [ ]:
# 購買履歴を結合してmissingを0埋めすることで売り上げがあるかないかのフラグにする
df_tmp = @linq leftjoin(df_customer, df_ifpurchased, on=:customer_id) |>
    transform(if_purchased = coalesce.(:if_purchased, 0));

In [ ]:
# 区分ごとの件数を見る
@linq df_tmp |>
    groupby(:if_purchased) |>
    combine(:customer_id => length)

In [ ]:
# MLDataPatternモジュールにお任せでundersample。ただ、デフォルトではエントリが縦方向に入っている（ふつうと逆！）想定なので
# obddim=1を指定して通常の縦長のテーブルであることを示す。
arr_sampled, _ = undersample((Array(df_tmp), df_tmp.if_purchased), obsdim=1)
df_sampled = DataFrame(arr_sampled)
rename!(df_sampled, names(df_tmp));

In [ ]:
# 件数が揃っていることを確認
@linq df_sampled |>
    groupby(:if_purchased) |>
    combine(:customer_id => length)

### 092

In [ ]:
df_gender = @linq df_customer |>
    select(:gender_cd, :gender) |>
    unique()
df_customer_s = @linq df_customer |>
    select(Not("gender"));

### 093

In [ ]:
# Notの引数ではSymbolが使えないので文字列で渡す
df_product_full = innerjoin(df_product, @select(df_category, Not(["category_major_cd", "category_medium_cd"])), on=:category_small_cd);
first(df_product_full, 3)

### 094

In [ ]:
# CSVモジュールにお任せ。デフォルトの設定でOK
CSV.write("data/category_julia.csv", df_product_full)

### 095

In [ ]:
# 簡単にはできなそうだし、現実的に使うとも思えないのでスキップ

### 096

In [ ]:
CSV.write("data/category_julia2.csv", df_product_full, header=false)

### 097

In [ ]:
# 保存する時と書き方が対照的でないので戸惑うが、CSV.read()はdeprecatedなのでこちらで
df_read = DataFrame!(CSV.File("data/category_julia.csv"));
first(df_read, 3)

### 098

In [ ]:
df_read2 = DataFrame!(CSV.File("data/category_julia2.csv"));
first(df_read2, 3)

### 099

In [ ]:
CSV.write("data/category_julia.tsv", df_product_full, delim="\t")

### 100

In [ ]:
# delim="\t"を指定してもいいが、ここでは自動判定に任せる
df_read3 = DataFrame!(CSV.File("data/category_julia.tsv"));
first(df_read3)

# これで１００本終わりです。おつかれさまでした！